# Training `prot_t5_xl_uniref50` model for predicting protein secondary structure using SageMaker Model Parallel 

## Install packages

*Note:* we only install the required libraries from Hugging Face and AWS. You also need PyTorch, so make sure, you are using `PyTorch 1.12 Python 3.8 CPU Optimized` kernel and `ml.t3.medium` instance to run this notebook. 

In [1]:
import sys

In [ ]:
!{sys.executable} -m pip install "sagemaker>=2.48.0" "transformers>=4.12.3" "datasets" --upgrade
!{sys.executable} -m pip install ipywidgets
!{sys.executable} -m pip install s3fs


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


## Development environment 

In [ ]:
import sagemaker.huggingface
import datasets

## Permissions

_If you are going to use Sagemaker in a local environment. You need access to an IAM Role with the required permissions for Sagemaker. You can find [here](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) more about it._

In [ ]:
import sagemaker

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::670488263423:role/sm-emr-SageMakerExecutionRole-LRXNQUL7LL5E
sagemaker bucket: sagemaker-us-east-1-670488263423
sagemaker session region: us-east-1


# Preprocessing

## Tokenization 

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

# dataset used
dataset_name = 'agemagician/NetSurfP-SS3'

# s3 key prefix for the data
s3_prefix = 'samples/datasets/netsurfp-ss3'

In [ ]:
# load dataset
dataset = load_dataset(dataset_name)

Using custom data configuration agemagician--NetSurfP-SS3-9c6b828487bb9f95
Found cached dataset parquet (/root/.cache/huggingface/datasets/agemagician___parquet/agemagician--NetSurfP-SS3-9c6b828487bb9f95/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

## Uploading data to `sagemaker_session_bucket`

After we processed the `datasets` we are going to use the `S3FileSystem` [integration](https://huggingface.co/docs/datasets/filesystems.html) to upload our dataset to S3.

In [ ]:
storage_options = {"anon": True}  # for anonymous connection
# for private buckets, uncomment the following code and add your aws access key id and secret key.
# storage_options = {"key": aws_access_key_id, "secret": aws_secret_access_key}  
import botocore
s3_session = botocore.session.Session()
storage_options = {"session": s3_session}

In [ ]:
import s3fs
fs = s3fs.S3FileSystem(**storage_options)

In [ ]:
# save dataset to s3
data_input_path = f's3://{sess.default_bucket()}/{s3_prefix}/data1'
dataset.save_to_disk(data_input_path,fs=fs)

data_input_path

/opt/conda/lib/python3.8/site-packages/datasets/dataset_dict.py:1161: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


Saving the dataset (0/1 shards):   0%|          | 0/646 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/646 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10792 [00:00<?, ? examples/s]

's3://sagemaker-us-east-1-670488263423/samples/datasets/netsurfp-ss3/data1'

In [ ]:
fs.ls(data_input_path)

['sagemaker-us-east-1-670488263423/samples/datasets/netsurfp-ss3/data1/dataset_dict.json',
 'sagemaker-us-east-1-670488263423/samples/datasets/netsurfp-ss3/data1/test',
 'sagemaker-us-east-1-670488263423/samples/datasets/netsurfp-ss3/data1/train',
 'sagemaker-us-east-1-670488263423/samples/datasets/netsurfp-ss3/data1/validation']

# Fine-tuning model using Sagemaker Training Job

In [ ]:
# training script
!pygmentize ./code/train.py

from __future__ import print_function

import glob
from itertools import chain
import os
import random
import zipfile
import argparse

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
from torchnet.dataset import SplitDataset

from datasets import load_from_disk
import requests
from tqdm.auto import tqdm
from seqeval.metrics import accuracy_score, f1_score, precision_score, recall_score
# from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import re
from transformers import AutoTokenizer, T5ForConditionalGeneration, AutoModelForTokenClassification, BertTokenizerFast, EvalPrediction, T5Tokenizer
from transfor

In [ ]:
model_name = "Rostlab/prot_t5_xl_uniref50"

## Creating an Estimator and start a training job

In [ ]:
from sagemaker.huggingface import HuggingFace

# hyperparameters, which are passed into the training job
hyperparameters={'epochs': 1,
                 'train_batch_size': 1,
                 'model_name': model_name
                 }

# configuration for running training on smdistributed Model Parallel
mpi_options = {
    "enabled" : True,
    "processes_per_host" : 8,
}
smp_options = {
    "enabled":True,
    "parameters": {
        "microbatches": 1,
        "placement_strategy": "cluster",
        "pipeline": "interleaved",
        "optimize": "memory",
        "partitions": 4,
        "ddp": True,
        # "tensor_parallel_degree": 2,
        "shard_optimizer_state": True,
        "activation_checkpointing": True, 
        "activation_strategy": "each",
        "activation_offloading": True,
    }
}

distribution={
    "smdistributed": {"modelparallel": smp_options},
    "mpi": mpi_options
}

In [ ]:
huggingface_estimator = HuggingFace(entry_point='train.py',
                            source_dir='./code',
                            instance_type='ml.p3dn.24xlarge',
                            instance_count=1,
                            role=role,
                            transformers_version='4.12',
                            pytorch_version='1.9',
                            py_version='py38',
                            distribution= distribution,
                            hyperparameters = hyperparameters,
                            keep_alive_period_in_seconds=60*60) # managed warm pool for 60mins

In [ ]:
# starting the train job with our uploaded datasets as input
huggingface_estimator.fit({'data': data_input_path})

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-pytorch-training-2022-12-19-22-03-50-459


2022-12-19 22:03:51 Starting - Starting the training job...
2022-12-19 22:04:07 Starting - Preparing the instances for training.....[1,mpirank:0,algo-1]<stderr>:#015  1%|          | 2/336 [01:18<3:32:45, 38.22s/it]
[1,mpirank:0,algo-1]<stderr>:#015 30%|██▉       | 100/336 [56:16<2:12:42, 33.74s/it]
[1,mpirank:0,algo-1]<stderr>:
[1,mpirank:0,algo-1]<stdout>:{'loss': 668985.68, 'learning_rate': 1.455e-05, 'epoch': 0.59}
[1,mpirank:0,algo-1]<stderr>:#015 30%|██▉       | 100/336 [56:16<2:12:42, 33.74s/it]
[1,mpirank:0,algo-1]<stderr>:***** Running Evaluation *****
[1,mpirank:0,algo-1]<stdout>:***** Running Evaluation *****
[1,mpirank:0,algo-1]<stderr>:  Num examples = 646
[1,mpirank:0,algo-1]<stderr>:  Batch size = 1
[1,mpirank:0,algo-1]<stdout>:  Num examples = 646
[1,mpirank:0,algo-1]<stdout>:  Batch size = 1
[1,mpirank:0,algo-1]<stderr>:
[1,mpirank:0,algo-1]<stderr>:#015  0%|          | 0/323 [00:00<?, ?it/s]#033[A
[1,mpirank:0,algo-1]<stderr>:
[1,mpirank:0,algo-1]<stderr>:#015  1%|    

## Deploying the endpoint

To deploy our endpoint, we call `deploy()` on our HuggingFace estimator object, passing in our desired number of instances and instance type.

In [ ]:
predictor = huggingface_estimator.deploy(1,"ml.g4dn.xlarge")

Finally, we delete the endpoint again.

In [ ]:
predictor.delete_endpoint()

# References